In [1]:
import os.path
import sys
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
import scoping
from utils.text import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC

import pandas as pd
import numpy as np

import pickle
import scipy.sparse

import matplotlib.pyplot as plt

In [2]:
seen_df = pd.read_csv('../data/0_labelled_documents.csv')
unseen_df = pd.read_csv('../data/0_unlabelled_documents.csv')

df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index

/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
f'../data/X_{df.shape[0]}.npz'

'../data/X_600690.npz'

In [4]:
revectorize = True

X_exists = os.path.isfile(f'../data/X_{df.shape[0]}.npz')

if revectorize is True or X_exists is False:
    print("running vectorisation again")
    vec = TfidfVectorizer(
        ngram_range=(1,2),
        min_df=10, max_df=0.8, strip_accents='unicode', 
        max_features=20000,
        tokenizer=snowball_stemmer()
    )
    vec.fit(df.loc[seen_index,"content"].astype("str"))

    X = vec.transform(df['content'].astype("str"))   
    with open (f'../data/vec_{seen_df.shape[0]}.pickle','wb') as f:
        pickle.dump(vec, f)
    import scipy.sparse
    scipy.sparse.save_npz(f'../data/X_{df.shape[0]}.npz', X)
else:
    print("loading feature matrix")
    with open (f'../data/vec_{seen_df.shape[0]}.pickle','rb') as f:
        vec = pickle.load(f)
        X = scipy.sparse.load_npz(f'../data/X_{df.shape[0]}.npz')
        
X.shape

running vectorisation again


/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(600690, 7180)

In [7]:
from sklearn.model_selection import KFold

y = df['relevant']

kf = KFold(n_splits=10)
kfs = kf.split(X[seen_index],y[seen_index])
y_preds = []
for i, (k_train, k_test) in enumerate(kfs):
    print(i)
    i+=0
    clf = SVC(kernel='rbf',class_weight='balanced',probability=True, C=10)
    k_train = seen_index[k_train]
    clf.fit(X[k_train],y[k_train])
    y_preds.append(clf.predict_proba(X[unseen_index])[:,1])
    
y_preds = np.array(y_preds)
np.save("../data/y_preds.npz",y_preds)

0
1
2
3
4
5
6
7
8
9


In [8]:
y_preds = np.load("../data/y_preds.npz.npy")
mean_pred = np.mean(y_preds, axis=0)
std_pred = np.std(y_preds, axis=0)
preds_upper = np.minimum(mean_pred + std_pred, 1)
preds_lower = np.maximum(mean_pred - std_pred, 0)

In [9]:
df.loc[unseen_index,'0 - relevance - mean_prediction']= mean_pred
df.loc[unseen_index,'0 - relevance - std_prediction'] = std_pred
df.loc[unseen_index,'0 - relevance - lower_pred'] = preds_lower
df.loc[unseen_index,'0 - relevance - upper_pred'] = preds_upper

cols = [
    "id",
    "0 - relevance - mean_prediction",
    "0 - relevance - std_prediction",
    "0 - relevance - lower_pred",
    "0 - relevance - upper_pred"  
]

df[cols].to_csv('../data/1_document_relevance.csv',index=False)

In [10]:
df

,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,physical_tags,...,8 - 8.09. Mixed methods,8 - 8.10. Case studies,8 - 8.11. Remote sensing,9 - observations,6 - 72 Conflict,4 - 50 Other (marine & coastal),0 - relevance - mean_prediction,0 - relevance - std_prediction,0 - relevance - lower_pred,0 - relevance - upper_pred
0,3921033,Recovery of phytosterol from roselle (Hibiscus...,Optimization of supercritical fluid extraction...,NaN,NaN,NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.262942,0.019405,0.243536,0.282347
1,2351389,Science denialism retards evidenced-based poli...,On allegations of invasive species denialism,discourse; facts; invasion biology; normative;...,['Biodiversity Conservation; Ecology; Environm...,NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.065476,0.009918,0.055558,0.075394
2,1686287,The QuikSCAT mission provided valuable daily i...,Wind characteristics in the North and Baltic S...,QuikSCAT; offshore wind energy; in situ; North...,"['Energy & Fuels; Engineering, Mechanical']",NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.040167,0.010916,0.029250,0.051083
3,1310127,This paper is to promote a further understandi...,The interdecadal variations of the summer mons...,NaN,['Meteorology & Atmospheric Sciences'],NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.320405,0.035234,0.285171,0.355639
4,1890786,Instrumental climate records are too short to ...,Corals as climate archive,NaN,"['Environmental Sciences; Geosciences, Multidi...",NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.774876,0.019936,0.754940,0.794812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600685,2331131,Perennial plant production and landscape utili...,Breeding and development of new ornamental pla...,Asclepias; Baptisia; Echinacea; Liatris; Penst...,['Biotechnology & Applied Microbiology; Hortic...,NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.143263,0.031245,0.112018,0.174508
600686,3825472,Ecoacoustics explains how animal and human beh...,Using acoustic perception to water sounds in t...,Urban; Noise; Comfort; Planning; Garden; Water,['Construction & Building Technology; Engineer...,NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.112108,0.013909,0.098199,0.126017
600687,3633258,Regulatory policy toward tobacco significantly...,Looking Back and Ahead: The Food and Drug Admi...,NaN,NaN,NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.191822,0.029090,0.162732,0.220912
600688,3825964,"Here, we report on a new collection of mostly ...","New fossils of Mesopithecus from Hasnot, Pakistan",Colobine; Cercopithecoid; Potwar; Miocene,['Anthropology; Evolutionary Biology'],NaN,0.0,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.100159,0.020040,0.080119,0.120199
